### Analyzing the Data
To pick up where I left off is my last notebook, I will start by loading in the table of files I created.

In [1]:
import pandas as pd

In [2]:
# Get files for 2017 to present
file_info = pd.read_csv('file_info.csv', dtype='str')
file_info = file_info[file_info['year'] >= '2017']
file_info.head()

,source,month,year,url,filename
0,yellow,01,2021,https://s3.amazonaws.com/nyc-tlc/trip+data/yel...,yellow-2021-01.csv
1,green,01,2021,https://s3.amazonaws.com/nyc-tlc/trip+data/gre...,green-2021-01.csv
2,fhv,01,2021,https://nyc-tlc.s3.amazonaws.com/trip+data/fhv...,fhv-2021-01.csv
3,fhvhv,01,2021,https://nyc-tlc.s3.amazonaws.com/trip+data/fhv...,fhvhv-2021-01.csv
4,yellow,02,2021,https://s3.amazonaws.com/nyc-tlc/trip+data/yel...,yellow-2021-02.csv


#### Exploring Data for Yellow Cabs
I wanted to get a sense of the data I was working with before processing it in large batches, so I did a little exploring with the first file downloaded, which was for yellow cab's January 2021.

In [3]:
yellow_cab = pd.read_csv('data/'+
                         file_info[file_info['source']=='yellow']['filename'][0], 
                         low_memory=False)

In [4]:
yellow_cab.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1369765 entries, 0 to 1369764
Data columns (total 18 columns):
VendorID                 1271413 non-null float64
tpep_pickup_datetime     1369765 non-null object
tpep_dropoff_datetime    1369765 non-null object
passenger_count          1271413 non-null float64
trip_distance            1369765 non-null float64
RatecodeID               1271413 non-null float64
store_and_fwd_flag       1271413 non-null object
PULocationID             1369765 non-null int64
DOLocationID             1369765 non-null int64
payment_type             1271413 non-null float64
fare_amount              1369765 non-null float64
extra                    1369765 non-null float64
mta_tax                  1369765 non-null float64
tip_amount               1369765 non-null float64
tolls_amount             1369765 non-null float64
improvement_surcharge    1369765 non-null float64
total_amount             1369765 non-null float64
congestion_surcharge     1369765 non-null fl

Let's take a look at the most and least expensive trip:

In [5]:
# Most expensive trip
max_fare = yellow_cab['total_amount'].max()
max_fare

7661.28

Now I want to find out what zones ths was between, so I will use the zone lookup table.

In [6]:
# Load zone lookup table
zones = pd.read_csv('https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv')

In [7]:
# Find the index of the max fare, and then lookup the zones, and then the borough
index = yellow_cab['total_amount'].idxmax()
PULocationID = yellow_cab.iloc[index]['PULocationID']
DOLocationID = yellow_cab.iloc[index]['DOLocationID']
PU_zone_name = zones.loc[PULocationID]['Zone']
DO_zone_name = zones.loc[DOLocationID]['Zone']
print(f'The most expensive fare of ${max_fare:.2f} was between {PU_zone_name} and {DO_zone_name}.')

The most expensive fare of $7661.28 was between Lenox Hill West and Upper East Side South.


Now I want to determine the least expensive trip.

In [8]:
# Least expensive trip
yellow_cab['total_amount'].min()

-492.8

While the most expensive trip is a bit shocking, the minimum trip is problematic. This data contains plenty of data on refunds, so I will need to filter out some of the data. Looking at [Taxi Fares](https://www1.nyc.gov/site/tlc/passengers/taxi-fare.page), it looks like the minimum fare is $2.50, so I will filter out fares less than this.

In [9]:
# Now to find the minimum fare and see the zones.
min_fare = yellow_cab[yellow_cab['total_amount'] >= 2.5]['total_amount'].min()
index = yellow_cab['total_amount'].idxmin()
PULocationID = yellow_cab.iloc[index]['PULocationID']
DOLocationID = yellow_cab.iloc[index]['DOLocationID']
PU_zone_name = zones.loc[PULocationID]['Zone']
DO_zone_name = zones.loc[DOLocationID]['Zone']
print(f'The least expensive fare of ${min_fare:.2f} was between {PU_zone_name} and {DO_zone_name}.')

The least expensive fare of $2.50 was between Upper East Side South and Upper East Side South.


Now that I have the range of fares I want to look at, I will calculate the total fares for this month's data. This will be useful later when looking at the trends of the fares over time.

In [10]:
# Calculate the sum of the fares, include refunds this time too since it matters for month's total.
sum_fares = yellow_cab['total_amount'].sum()
print(f'Fares for the month total ${sum_fares:,.0f}.')

Fares for the month total $23,935,806.


Now I want to look at the most popular payment method. Looking at the [Data Dictionary](https://www1.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf) for the Yellow Cabs, it looks like the payment types are:
1. Credit card
2. Cash
3. No charge
4. Dispute
5. Unknown
6. Voided trip
I will look at credit cards and cash only, since types 3-6 are not actual trips.

In [11]:
# Filter for payment types 1 and 2 and then count each
no_ccs = yellow_cab[yellow_cab['payment_type']==1]['payment_type'].count()
no_cash = yellow_cab[yellow_cab['payment_type']==2]['payment_type'].count()
print(f'There were {no_ccs:,.0f} credit card and {no_cash:,.0f} cash transactions this month.')

There were 934,473 credit card and 322,889 cash transactions this month.


Credit card was about 3 times more popular than cash. 

Now let's look at the most expensive days to travel. First, I want to tag each trip with the day of the week.

In [12]:
# Convert date column to datetime format to get the actual days of the trip
yellow_cab['tpep_pickup_datetime'] = pd.to_datetime(yellow_cab['tpep_pickup_datetime'])
day_names = ['Mon', 'Tues', 'Wed', 'Thurs', 'Fri', 'Sat', 'Sun']
days = [day_names[day.weekday()] for day in yellow_cab['tpep_pickup_datetime']]
yellow_cab['day'] = days
yellow_cab.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,day
0,1.0,2021-01-01 00:30:10,2021-01-01 00:36:12,1.0,2.10,1.0,N,142,43,2.0,8.0,3.0,0.5,0.00,0.0,0.3,11.80,2.5,Fri
1,1.0,2021-01-01 00:51:20,2021-01-01 00:52:19,1.0,0.20,1.0,N,238,151,2.0,3.0,0.5,0.5,0.00,0.0,0.3,4.30,0.0,Fri
2,1.0,2021-01-01 00:43:30,2021-01-01 01:11:06,1.0,14.70,1.0,N,132,165,1.0,42.0,0.5,0.5,8.65,0.0,0.3,51.95,0.0,Fri
3,1.0,2021-01-01 00:15:48,2021-01-01 00:31:01,0.0,10.60,1.0,N,138,132,1.0,29.0,0.5,0.5,6.05,0.0,0.3,36.35,0.0,Fri
4,2.0,2021-01-01 00:31:49,2021-01-01 00:48:21,1.0,4.94,1.0,N,68,33,1.0,16.5,0.5,0.5,4.06,0.0,0.3,24.36,2.5,Fri


Now, I want to calculate the average fare for each day of the month.

In [13]:
for day in day_names:
    avg_fare = yellow_cab[yellow_cab['day']==day]['total_amount'].mean()
    print(f'The average fare for {day} was ${avg_fare:.2f}.')

The average fare for Mon was $17.72.
The average fare for Tues was $17.32.
The average fare for Wed was $17.07.
The average fare for Thurs was $17.36.
The average fare for Fri was $17.39.
The average fare for Sat was $17.36.
The average fare for Sun was $18.39.


Sunday was the most expensive day for this month. 

Now that I have figured out how to obtain this data for a single month, I will iterate this for each month and then visualize the data. I will write a function to help automate this. 

In [96]:
# This function will take a filename and return all the stats calculate before for a month's dataset.
def get_stats(filename):
    
    # load data
    df = pd.read_csv('data/'+filename, low_memory=False)
    
    # for debugging
    if 'fhv' in filename:
        print(filename)

    # load zone lookup table
    zones = pd.read_csv('https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv')

    stats = pd.DataFrame()
    
    if 'yellow' in filename or 'green' in filename:
        df = df[df['PULocationID'] < 264] # Get rid of N/A zones
        df = df[df['DOLocationID'] < 264] # Get rid of N/A zones
        df = df[df['payment_type'] < 3].reset_index(drop=True) # Get rid of non-payments
        
        # Calculate Max fare info
        stats['max_fare'] = [float(df['total_amount'].max())]
        index = df['total_amount'].idxmax()
        PULocationID_max = df.iloc[index]['PULocationID']
        DOLocationID_max = df.iloc[index]['DOLocationID']
        stats['PU_zone_name_max'] = [zones.loc[PULocationID_max]['Zone']]
        stats['DO_zone_name_max'] = [zones.loc[DOLocationID_max]['Zone']]

        # Calculate Min fare info
        stats['min_fare'] = [df[df['total_amount'] >= 2.5]['total_amount'].min()]
        index = df['total_amount'].idxmin()
        PULocationID_min = df.iloc[index]['PULocationID']
        DOLocationID_min = df.iloc[index]['DOLocationID']
        stats['PU_zone_name_min'] = [zones.loc[PULocationID_min]['Zone']]
        stats['DO_zone_name_min'] = [zones.loc[DOLocationID_min]['Zone']]

        # Calculate month's total rake
        stats['sum_fares'] = [df['total_amount'].sum()]

        # Filter for payment types 1 and 2 and then count each
        stats['num_ccs'] = [df[df['payment_type']==1]['payment_type'].count()]
        stats['num_cash'] = [df[df['payment_type']==2]['payment_type'].count()]
    
    # Calculate averages for each day of the week
    day_names = ['Mon', 'Tues', 'Wed', 'Thurs', 'Fri', 'Sat', 'Sun']
    
    # Have to take into account different columns names for the different sources
    if 'yellow' in filename:
        df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
        days = [day_names[day.weekday()] for day in df['tpep_pickup_datetime']]
    elif 'green' in filename:
        df['lpep_pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'])
        days = [day_names[day.weekday()] for day in df['lpep_pickup_datetime']]
    else:
        df = df.dropna(subset=['pickup_datetime'])
        df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
        days = [day_names[day.weekday()] for day in df['pickup_datetime']]    
    
    df['day'] = days

    
    day_avgs = []
    day_counts = []
    
    for day in day_names:
        if 'yellow' in filename or 'green' in filename:
            avg_fare = df[df['day']==day]['total_amount'].mean()
            day_avgs.append(avg_fare)
            stats['day_avgs']  = [day_avgs]
        else:
            count_fare = df[df['day']==day]['pickup_datetime'].count()
            day_counts.append(count_fare)
            stats['day_counts']  = [count_fare]
    
    return stats
    

Compiling the statistics now for all the yellow cab data:

In [99]:
yellow_files = file_info[file_info['source']=='yellow']['filename'].reset_index(drop=True)
yellow_stats = get_stats(yellow_files[0])

for file in yellow_files[1:]:
    yellow_stats = yellow_stats.append(get_stats(file))

yellow_stats.head()

,max_fare,PU_zone_name_max,DO_zone_name_max,min_fare,PU_zone_name_min,DO_zone_name_min,sum_fares,num_ccs,num_cash,day_avgs
0,7661.28,Lenox Hill West,Upper East Side South,2.5,Kensington,Kensington,20337832.22,923615,317996,"[16.515076701184835, 16.097442475670412, 15.93..."
0,6969.30,Midtown East,Two Bridges/Seward Park,2.5,Astoria Park,Astoria Park,20581962.71,939415,304856,"[16.378399147558518, 16.381276154527242, 16.69..."
0,6010.80,Upper West Side South,Central Harlem North,2.5,Laurelton,Laurelton,29301287.06,1331089,425475,"[16.636681144482775, 16.496921545365453, 16.47..."
0,2059.30,Yorkville West,Yorkville West,2.5,University Heights/Morris Heights,East Concourse/Concourse Village,34830185.40,1534333,463029,"[17.481425546301512, 17.35579707665767, 17.220..."
0,1165.30,Marine Park/Mill Basin,Brighton Beach,2.5,East Concourse/Concourse Village,East Concourse/Concourse Village,42215363.41,1809267,521178,"[18.283418543805897, 17.768654668323805, 17.91..."


In [100]:
yellow_file_data = file_info[file_info['source']=='yellow'].reset_index(drop=True)
yellow_stats = yellow_stats.reset_index(drop=True).join(yellow_file_data)
yellow_stats.to_csv('yellow_stats.csv', index=False)

And now the green data:

In [97]:
green_files = file_info[file_info['source']=='green']['filename'].reset_index(drop=True)
green_stats = get_stats(green_files[0])

for file in green_files[1:]:
    green_stats = green_stats.append(get_stats(file))
    
green_stats.head()

,max_fare,PU_zone_name_max,DO_zone_name_max,min_fare,PU_zone_name_min,DO_zone_name_min,sum_fares,num_ccs,num_cash,day_avgs
0,293.50,Stapleton,Stapleton,2.50,Corona,Norwood,692416.19,24060,15489,"[17.52057696030978, 17.790914950760968, 17.809..."
0,207.20,Corona,Broad Channel,3.30,Randalls Island,Randalls Island,626725.24,21591,13119,"[17.99955420466059, 18.147730086669416, 18.567..."
0,483.06,DUMBO/Vinegar Hill,East Flatbush/Remsen Village,3.05,Astoria Park,Astoria Park,759199.92,25413,16918,"[18.300199549887473, 18.190173640762882, 18.02..."
0,411.04,Central Park,Stapleton,2.80,West Brighton,East Elmhurst,834182.72,26871,18037,"[18.90855790741697, 18.977560429005926, 18.606..."
0,2113.55,Bedford Park,Far Rockaway,2.50,Crown Heights North,Seaport,921321.90,30398,18978,"[18.573577958027553, 19.357118978878592, 18.82..."


In [98]:
green_file_data = file_info[file_info['source']=='green'].reset_index(drop=True)
green_stats = green_stats.reset_index(drop=True).join(green_file_data)
green_stats.to_csv('green_stats.csv', index=False)

I chose not to use the FHV and FHV High Volume data since it does not contain fare information, which is necessary to calculare any of information I found for the yellow and green cabs. The next notebook is for visualizing this data, which can be found [HERE](Visualizations.ipynb). 